In [1]:
import random

In [2]:
with open('../sample_data/kin/train/train_data') as f:
    datas = [line.strip().split('\t') for line in f.readlines()]
with open('../sample_data/kin/train/train_label') as f:
    labels = [bool(int(line)) for line in f.readlines()]

In [3]:
from gensim.models import doc2vec
from gensim.models.doc2vec import LabeledSentence

In [4]:
from konlpy import tag

In [5]:
tagger = tag.Twitter()

In [6]:
ignore = ['Josa', 'Punctuation']

In [7]:
def parse(sentence):
    return [word for word, tag in tagger.pos(sentence) if tag not in ignore]

In [8]:
docs = []
for i, line in enumerate(datas):
    docs.append(LabeledSentence(parse(line[0]), ['SENT_' + str(i * 2)]))
    docs.append(LabeledSentence(parse(line[1]), ['SENT_' + str(i * 2 + 1)]))

In [9]:
model = doc2vec.Doc2Vec(size=100, window=300, min_count=1, workers=4)

In [10]:
model.build_vocab(docs)

In [11]:
val_alpha = .025
min_alpha = 1e-4
passes = 1000
del_alpha = (val_alpha - min_alpha) / (passes - 1)

In [12]:
for epoch in range(passes):
    random.shuffle(docs)
    model.alpha, model.min_alpha = val_alpha, val_alpha
    model.train(docs)
    if not epoch % 100:
        print('Completed pass %i at alpha %f' % (epoch + 1, val_alpha))
    val_alpha -= del_alpha

Completed pass 1 at alpha 0.025000
Completed pass 101 at alpha 0.022508
Completed pass 201 at alpha 0.020015
Completed pass 301 at alpha 0.017523
Completed pass 401 at alpha 0.015030
Completed pass 501 at alpha 0.012538
Completed pass 601 at alpha 0.010045
Completed pass 701 at alpha 0.007553
Completed pass 801 at alpha 0.005060
Completed pass 901 at alpha 0.002568


In [13]:
from scipy import spatial

In [14]:
for i in range(5):
    print (1 - spatial.distance.cosine(model.docvecs[0], model.docvecs[i * 2 + 1]), labels[i])

0.519368111592 True
0.399751373458 False
0.533336276693 False
0.293422340263 False
0.398615664346 False


In [54]:
1-spatial.distance.cosine(model.docvecs[0], model.docvecs[100])

0.35211880427275122

In [19]:
import numpy as np

In [21]:
features = model.docvecs

# for this example we're randomly sampling 5M of them, so here I sample and save the random indices of the sample
random_indices = np.random.choice(range(5000000),replace=False)
features = features[random_indices]

# now we write the data to file in the required LargeVis format (which requires a header with the number of items and the dimensionality of the feature vectors)
with open('lv_format.txt','w') as out:
    out.write("{}\t{}\n".format(*features.shape))
    for row in features:
        out.write('\t'.join(row.astype(str))+'\n')

AttributeError: 'numpy.int64' object has no attribute 'asype'